## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,69.91,84,24,2.26,few clouds
1,1,Schladming,AT,47.3929,13.6870,35.53,99,99,5.14,light snow
2,2,Hervey Bay,AU,-25.2986,152.8535,83.03,78,100,12.50,overcast clouds
3,3,Kodiak,US,57.7900,-152.4072,38.71,100,100,9.22,mist
4,4,George,ZA,-33.9630,22.4617,74.10,76,15,1.66,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
vacation_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hervey Bay,AU,-25.2986,152.8535,83.03,78,100,12.50,overcast clouds
18,18,Butaritari,KI,3.0707,172.7902,81.55,76,55,10.45,broken clouds
26,26,Castro,BR,-24.7911,-50.0119,75.72,72,4,4.79,clear sky
28,28,Georgetown,MY,5.4112,100.3354,82.33,86,20,2.30,few clouds
29,29,Makakilo City,US,21.3469,-158.0858,79.12,54,0,11.50,clear sky
31,31,Hilo,US,19.7297,-155.0900,78.53,70,20,5.75,light rain
42,42,East London,ZA,-33.0153,27.9116,75.81,87,100,6.08,overcast clouds
43,43,Hithadhoo,MV,-0.6000,73.0833,82.45,72,5,11.39,clear sky
45,45,Kupang,ID,-10.1667,123.5833,79.21,84,10,10.29,clear sky
46,46,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,80.58,75,32,7.20,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.notnull().sum()

City_ID                199
City                   199
Country                199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [6]:
vacation_cities_df.count()

City_ID                199
City                   199
Country                199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = vacation_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hervey Bay,AU,83.03,overcast clouds,-25.2986,152.8535,
18,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,
26,Castro,BR,75.72,clear sky,-24.7911,-50.0119,
28,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
29,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,
31,Hilo,US,78.53,light rain,19.7297,-155.0900,
42,East London,ZA,75.81,overcast clouds,-33.0153,27.9116,
43,Hithadhoo,MV,82.45,clear sky,-0.6000,73.0833,
45,Kupang,ID,79.21,clear sky,-10.1667,123.5833,
46,Sao Jose Da Coroa Grande,BR,80.58,scattered clouds,-8.8978,-35.1478,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hervey Bay,AU,83.03,overcast clouds,-25.2986,152.8535,Shelly Bay Resort
18,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,Isles Sunset Lodge
26,Castro,BR,75.72,clear sky,-24.7911,-50.0119,CHACARA BAILLY
28,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
29,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
31,Hilo,US,78.53,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
42,East London,ZA,75.81,overcast clouds,-33.0153,27.9116,Tu Casa
43,Hithadhoo,MV,82.45,clear sky,-0.6000,73.0833,Scoop Guest House
45,Kupang,ID,79.21,clear sky,-10.1667,123.5833,Hotel On The Rock by Prasanthi
46,Sao Jose Da Coroa Grande,BR,80.58,scattered clouds,-8.8978,-35.1478,Crôa Mares Hotel


In [11]:
hotel_df.count()

City                   199
Country                199
Max Temp               199
Current Description    199
Lat                    199
Lng                    199
Hotel Name             199
dtype: int64

In [12]:
hotel_df.notnull().sum()

City                   199
Country                199
Max Temp               199
Current Description    199
Lat                    199
Lng                    199
Hotel Name             199
dtype: int64

In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [17]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hervey Bay,AU,83.03,overcast clouds,-25.2986,152.8535,Shelly Bay Resort
18,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,Isles Sunset Lodge
26,Castro,BR,75.72,clear sky,-24.7911,-50.0119,CHACARA BAILLY
28,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
29,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.info_box_template = """
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}<dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.05)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))